# part0: imports

In [2]:
import os, sys, pathlib
from pprint import pprint 
from importlib import reload
import logging
from typing import Callable
import warnings
import pandas as pd
import numpy as np
import xarray as xr
from sklearn.decomposition import PCA
import scipy.linalg as linalg

import matplotlib.pyplot as plt
import matplotlib.cm as cm
import matplotlib
from matplotlib.ticker import MaxNLocator


import pyaldata as pyal

%matplotlib inline

try:
    nbPath = pathlib.Path.cwd()
    RepoPath = nbPath.parent
    os.chdir(RepoPath)

    from tools import utilityTools as utility
    from tools import dataTools as dt
    import params
    defs = params.random_walk_defs
    
    set_rc =  params.set_rc_params
    root = params.root

finally:
    os.chdir(nbPath)

warnings.filterwarnings('ignore')

/home/jcc319/anaconda3/envs/multi-animal-alignment/lib/python3.10/site-packages/tqdm/auto.py:22: TqdmWarning: IProgress not found. Please update jupyter and ipywidgets. See https://ipywidgets.readthedocs.io/en/stable/user_install.html
  from .autonotebook import tqdm as notebook_tqdm


#### Print the summary of all the datassets

check out the results [here](https://github.com/AtMostafa/notebook/blob/main/2021-cca-project/dataset-summary.md).

In [3]:
datadir = root/'random_walk'
_AllAnimalList = ['Chewie', 'Mihili', "MrT"]

_AllAnimalFiles=[]
for animal in _AllAnimalList:
    _AllAnimalFiles.extend(utility.find_file(datadir / animal, 'mat'))
# print(_AllAnimalFiles)


In [17]:
rows = []
for file in _AllAnimalFiles:
    raw_df = pyal.mat2dataframe(file, shift_idx_fields=True)
    df = defs.prep_general(raw_df)

    dic = {
        'animal': file.split(os.sep)[-2],
        'file': file.split(os.sep)[-1],
        'areas': [x for x in list(raw_df.columns) if '_spikes' in x],
        'trials_all': len(raw_df),
        'trials_left': len(df),
        'neurons_all': [raw_df[x][0].shape[1] for x in list(raw_df.columns) if '_spikes' in x],
        'neurons_left': [df[x][0].shape[1] for x in list(df.columns) if '_spikes' in x]
    }
    rows.append(dic)

summary = pd.DataFrame(rows)
summary


,animal,file,areas,trials_all,trials_left,neurons_all,neurons_left
0,Chewie,Chewie_RT_VR_2013-12-13.mat,[M1_spikes],486,157,[67],[54]
1,Chewie,Chewie_RT_VR_2013-10-11.mat,[M1_spikes],509,134,[88],[65]
2,Chewie,Chewie_RT_FF_2013-12-09.mat,[M1_spikes],529,148,[65],[48]
3,Chewie,Chewie_RT_FF_2013-12-18.mat,[M1_spikes],588,154,[62],[53]
4,Chewie,Chewie_RT_FF_2013-12-17.mat,[M1_spikes],634,157,[57],[45]
5,Chewie,Chewie_RT_VR_2013-12-12.mat,[M1_spikes],553,164,[61],[39]
6,Chewie,Chewie_RT_FF_2013-10-28.mat,[M1_spikes],530,135,[77],[49]
7,Chewie,Chewie_RT_FF_2013-12-10.mat,[M1_spikes],628,159,[63],[42]
8,Chewie,Chewie_RT_VR_2013-10-10.mat,[M1_spikes],577,147,[74],[31]
9,Chewie,Chewie_RT_CS_2016-10-21.mat,"[M1_spikes, PMd_spikes]",291,288,"[84, 211]","[84, 196, 280]"


In [16]:
raw_df = pyal.mat2dataframe(_AllAnimalFiles[9], shift_idx_fields=True)
raw_df.M1_spikes.values[0].shape, raw_df.PMd_spikes.values[0].shape, raw_df.pos.values[0].shape,



((347, 84), (347, 211), (347, 2))

Categories I want to separate:

- sessions with 2 areas, similar number of neurons each, as many monkeys as possible!
- 3 sessions for each area (M1, PMd, S1), most number of neurons

In [12]:
_AllAnimalFiles= [path.split(os.sep)[-1] for path in _AllAnimalFiles]

The code above relies on the host PC having the exact fiels as my laptop.  
So I replace the values below for reliability.

In [25]:
GoodDataList = {'M1':{}, 'PMd':{}}
#-----------------------------------

for area in ['M1', 'PMd']:
    for animal in ['Chewie', 'Mihili', 'MrT']:
        area_idx = [area+'_spikes' in x for x in summary.areas]
        files = summary[area_idx & (summary.animal == animal)].file.values
        GoodDataList[area][animal] = files

GoodDataList

{'M1': {'Chewie': array(['Chewie_RT_VR_2013-12-13.mat', 'Chewie_RT_VR_2013-10-11.mat',
         'Chewie_RT_FF_2013-12-09.mat', 'Chewie_RT_FF_2013-12-18.mat',
         'Chewie_RT_FF_2013-12-17.mat', 'Chewie_RT_VR_2013-12-12.mat',
         'Chewie_RT_FF_2013-10-28.mat', 'Chewie_RT_FF_2013-12-10.mat',
         'Chewie_RT_VR_2013-10-10.mat', 'Chewie_RT_CS_2016-10-21.mat',
         'Chewie_RT_FF_2013-10-29.mat', 'Chewie_RT_VR_2013-10-09.mat'],
        dtype=object),
  'Mihili': array(['Mihili_RT_VR_2014-01-15.mat', 'Mihili_RT_FF_2014-02-14.mat',
         'Mihili_RT_FF_2014-02-24.mat', 'Mihili_RT_FF_2014-02-21.mat',
         'Mihili_RT_VR_2014-01-16.mat', 'Mihili_RT_VR_2014-01-14.mat'],
        dtype=object),
  'MrT': array(['MrT_RT_FF_2013-08-22.mat', 'MrT_RT_VR_2013-09-04.mat',
         'MrT_RT_VR_2013-09-06.mat', 'MrT_RT_VR_2013-09-10.mat',
         'MrT_RT_FF_2013-08-30.mat', 'MrT_RT_FF_2013-08-20.mat'],
        dtype=object)},
 'PMd': {'Chewie': array(['Chewie_RT_CS_2016-10-21.mat'], dt

Adding the aggregate of *M1* and *PMd* as **MCx**

In [26]:
MCx = {}
for area in ['M1', 'PMd']:
    for animal in GoodDataList[area]:
        if animal not in MCx:
            MCx[animal] = []
        MCx[animal].extend(GoodDataList[area][animal])
        MCx[animal] = list(set(MCx[animal]))

GoodDataList['MCx'] = MCx